In [2]:
import os
import gym
import pygame
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
env = gym.make("CartPole-v1", render_mode = "human")
episodes = 5

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done,truncated, info = env.step(action)
        score += reward

    print("Episode:{} Score: {}".format(episode, score))

pygame.quit()
env.close()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score: 19.0
Episode:2 Score: 11.0
Episode:3 Score: 13.0
Episode:4 Score: 16.0
Episode:5 Score: 12.0


In [3]:
log_path = os.path.join("training", "logs")

In [10]:
env = gym.make("CartPole-v1", render_mode = "human")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose = 1, tensorboard_log = log_path)

Using cpu device


In [ ]:
model.learn(total_timesteps = 10)

Logging to training/logs/PPO_2


In [6]:
PPO_path = os.path.join("training", "save models", "PPO_Model_Cartpole")
model.save(PPO_path)
del model
model = PPO.load(PPO_path, env=env)

In [7]:
model

In [9]:
evaluate_policy(model, env, n_eval_episodes = 10, render = True)

TypeError: evaluate_policy() got an unexpected keyword argument 'render_mode'

In [8]:
env.close()

In [10]:
episodes = 5

for episode in range(1, episodes+1):
    obs = env.reset()
    terminated = False
    score = 0
    while not terminated:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, terminated, info = env.step(action)
        score += reward
    print("Episode:{} Score: {}".format(episode, score))
env.close()

Episode:1 Score: [362.]
Episode:2 Score: [241.]
Episode:3 Score: [467.]
Episode:4 Score: [500.]
Episode:5 Score: [500.]


In [11]:
obs = env.reset()

In [12]:
model.predict(obs)

(array([0]), None)

In [13]:
env.action_space.sample()

0

In [14]:
env.step(action)

(array([[-0.04778304, -0.23889387, -0.00809708,  0.24992731]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [17]:
training_log_path = os.path.join(log_path, "PPO_9")
training_log_path

'training/logs/PPO_9'

In [18]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
%tensorboard --logdir={training_log_path}

In [20]:
!kill 24597

zsh:kill:1: kill 24597 failed: no such process


In [21]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join("training", "save models")
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path=save_path, verbose=1)

In [22]:
model = PPO("MlpPolicy", env, verbose =1, tensorboard_log = log_path)

Using cpu device


In [23]:
model.learn(total_timesteps = 20000, callback=eval_callback)

Logging to training/logs/PPO_10
-----------------------------
| time/              |      |
|    fps             | 6460 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 4510       |
|    iterations           | 2          |
|    time_elapsed         | 0          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00891187 |
|    clip_fraction        | 0.12       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.685     |
|    explained_variance   | 0.000949   |
|    learning_rate        | 0.0003     |
|    loss                 | 5.36       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0193    |
|    value_loss           | 50.6       |
----------------------------------------
--------------------

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=427.80 +/- 86.15
Episode length: 427.80 +/- 86.15
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 428          |
|    mean_reward          | 428          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0072104298 |
|    clip_fraction        | 0.0793       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.603       |
|    explained_variance   | 0.346        |
|    learning_rate        | 0.0003       |
|    loss                 | 14.1         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0198      |
|    value_loss           | 59.8         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 427.80  is above the threshold 200


In [24]:
 net_arch = [dict(pi = [128,128,128,128], vf = [128,128,128,128])]

In [25]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log = log_path, policy_kwargs = {"net_arch": net_arch})

Using cpu device


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/stable_baselines3/common/policies.py:454: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [28]:
model.learn(total_timesteps = 20000, callback=eval_callback)

Logging to training/logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 5373 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3000        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012880512 |
|    clip_fraction        | 0.211       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.0187     |
|    learning_rate        | 0.0003      |
|    loss                 | 2.82        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0236     |
|    value_loss           | 19.4        |
-----------------------------------------
---

In [29]:
from stable_baselines3 import DQN

In [30]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log = log_path)

Using cpu device


In [26]:
model.learn(total_timesteps = 20000, callback=eval_callback)

Logging to training/logs/PPO_11
-----------------------------
| time/              |      |
|    fps             | 5363 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3022        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013616152 |
|    clip_fraction        | 0.15        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.683      |
|    explained_variance   | 0.000427    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.06        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0176     |
|    value_loss           | 22.7        |
-----------------------------------------
--

In [31]:
training_log_path = os.path.join(log_path, "PPO_4")

%tensorboard --logdir={training_log_path}